In [3]:
# Uncomment the following lines to install the required packages
!pip install opendatasets #download the data set
!pip install pandas #data manipulation and analysis

Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip
Defaulting to user installation because normal site-packages is not writeable

[notice] A new release of pip is available: 23.3.2 -> 24.0
[notice] To update, run: /Library/Developer/CommandLineTools/usr/bin/python3 -m pip install --upgrade pip


In [64]:
import pandas as pd

df_cancer = pd.read_csv('breast_cancer.csv', encoding='cp1252', sep=',')

df_cancer

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Marginal Adhesion,Single Epithelial Cell Size,Bare Nuclei,Bland Chromatin,Normal Nucleoli,Mitoses,Class
0,5,1,1,1,2,1,3,1,1,2
1,5,4,4,5,7,10,3,2,1,2
2,3,1,1,1,2,2,3,1,1,2
3,6,8,8,1,3,4,3,7,1,2
4,4,1,1,3,2,1,3,1,1,2
...,...,...,...,...,...,...,...,...,...,...
678,3,1,1,1,3,2,1,1,1,2
679,2,1,1,1,2,1,1,1,1,2
680,5,10,10,3,7,3,8,10,2,4
681,4,8,6,4,3,4,10,6,1,4


In [65]:
# Select specific columns to clean the data
df_final = df_cancer[['Clump Thickness', 'Uniformity of Cell Size', 'Uniformity of Cell Shape', 'Single Epithelial Cell Size', 'Bare Nuclei', 'Normal Nucleoli', 'Mitoses']]

# Display the cleaned DataFrame
df_final

,Clump Thickness,Uniformity of Cell Size,Uniformity of Cell Shape,Single Epithelial Cell Size,Bare Nuclei,Normal Nucleoli,Mitoses
0,5,1,1,2,1,1,1
1,5,4,4,7,10,2,1
2,3,1,1,2,2,1,1
3,6,8,8,3,4,7,1
4,4,1,1,2,1,1,1
...,...,...,...,...,...,...,...
678,3,1,1,3,2,1,1
679,2,1,1,2,1,1,1
680,5,10,10,7,3,10,2
681,4,8,6,3,4,6,1


In [66]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from sklearn.linear_model import LogisticRegression

# Assuming df_new is the DataFrame with the features for prediction
x = df_final

# Use the 'Class' column as the target variable and convert it to binary
def replace_score(score):
    return 1 if score == 4 else 0

y = df_cancer['Class'].apply(replace_score)

# Split into testing and training datasets
X_train, X_test, y_train, y_test = train_test_split(x, y, test_size=0.2, random_state=42)

# Train a logistic regression model
model = LogisticRegression(max_iter=1000)  # Increase max_iter if needed for convergence
model.fit(X_train, y_train)

# Test the model and report accuracy
y_pred = model.predict(X_test)
accuracy = accuracy_score(y_test, y_pred)
print('Accuracy:', accuracy)

Accuracy: 0.9562043795620438


### Predicting Survival
So, now we are ready to play the game... "Would I have survived the Titanic?".  

Insert your own data in the code.  Look at your analysis and consider how you would travel today.
- Data description:
    - pclass - Passenger Class (1 = 1st; 2 = 2nd; 3 = 3rd)
    - name - Name
    - sex - male or female
    - age - number of year 
    - sibsp - number of Siblings/Spouses Aboard
    - parch - number of Parents/Children Aboard
    - fare - passenger fare 0 to 512
    - embarked - Port of Embarkation (C = Cherbourg; Q = Queenstown; S = Southampton)
    - alone - boolean True or False

In [1]:
import numpy as np
import pandas as pd

# Define a new patient's data as a DataFrame with the specific features from the breast cancer dataset
new_patient = pd.DataFrame({
    'Clump Thickness': [2],
    'Uniformity of Cell Size': [8],
    'Uniformity of Cell Shape': [1],
    'Single Epithelial Cell Size': [8],
    'Bare Nuclei': [6],
    'Normal Nucleoli': [1],
    'Mitoses': [1]
})

# Use the trained logistic regression model (logreg) to predict the probability
# of the tumor being benign (0) or malignant (1)
probability = logreg.predict_proba(new_patient)

# Extract the probabilities
benign_proba, malignant_proba = probability[0]

# Print the predicted probabilities for the tumor being benign and malignant
print('Benign probability: {:.2%}'.format(benign_proba))
print('Malignant probability: {:.2%}'.format(malignant_proba))

NameError: name 'logreg' is not defined

In [69]:
from joblib import dump, load

#Save file using joblib
dump(model, 'cancer_model.joblib') 

['cancer_model.joblib']

In [70]:
model = load('cancer_model.joblib')

In [71]:
import pandas as pd
import numpy as np

from flask_restful import Api, Resource
from flask import Blueprint, request
from joblib import load

#Load the ML model: replace the file name with whatever yours is
model = load('./api/cancer_model.joblib')

#Initialize Flask API endpoint blueprint
cancer_api = Blueprint('cancer_api', __name__, url_prefix='/api/cancer')
api = Api(cancer_api)

#Use a post request to take data from the frontend as a JSON, then return output value
class cancerAPI:
    class _Predict(Resource):
        def post(self):
            #Get data from frontend
            body = request.get_json()
            
            if body is not None:
                #Convert frontend JSON output to a pandas dataframe
                data = pd.DataFrame([body])
                
                #Predict and return the cancer score (model.predict returns a 1-element array so we need to take a slice)
                score = model.predict(data)[0]
                return {'score': score}, 200
            else:
                return {'message': 'No data provided'}, 400
    
    #Add endpoint resource for this method
    api.add_resource(_Predict, '/predict')

FileNotFoundError: [Errno 2] No such file or directory: './api/cancer_model.joblib'

In [53]:
from __init__ import app
from api.cancerr import cancer_api
from flask import Flask
from flask_cors import CORS

#Enable CORS for everything
app = Flask(__name__)
CORS(app)

app.register_blueprint(cancer_api)

#Allow all CORS headers before requests
@app.before_request
def before_request():
    allowed_origin = request.headers.get('Origin')
    if allowed_origin:
        cors._origins = "*"

ModuleNotFoundError: No module named '__init__'